In [3]:
import pandas as pd
import numpy as np
import glob
import re
from tqdm import tqdm
from sklearn.feature_extraction import DictVectorizer
# import enchant
from collections import Counter

In [8]:
csv_names = glob.glob('./data_new/*.csv')


In [9]:
df_from_each_file = (pd.read_csv(f,encoding = 'latin1') for f in csv_names)
data = pd.concat(df_from_each_file, ignore_index=True)
data.head()

,Record_id,y_pred,y_act,Reason,y_Arun,check,Attribute_name,Total_val,num of dist_val,Unnamed: 9,Num of nans,mean,std_dev,min_val,max_val,sample_1,sample_2,sample_3,sample_4,sample_5
0,33,Usable directly numeric,Usable directly categorical,g,Usable directly categorical,check,Area Code,21477,174,0.810169,0,125.449411,72.866452,1.0,276.0,2,3,4,7,8
1,33,Usable directly numeric,Context_specific,m,Usable directly categorical,check,Item Code,21477,117,0.544769,0,2694.211529,148.969938,2511.0,2961.0,2511,2805,2513,2514,2517
2,33,Usable directly numeric,Context_specific,m,Usable directly categorical,check,Element Code,21477,2,0.009312,0,5211.687154,146.816661,5142.0,5521.0,5142,5521,5142,5142,5521
3,33,Usable directly categorical,Usable directly categorical,h,Usable directly categorical,check,Element,21477,2,0.009312,0,0.000000,0.000000,0.0,0.0,Food,Feed,Food,Food,Food
4,37,Usable with extraction,Context_specific,n,Context_specific,check,Loan Theme ID,15736,718,4.562786,0,0.000000,0.000000,0.0,0.0,a1050000000slfi,a10500000068jPe,a1050000002X1Uu,a1050000007VvXr,a1050000000weyk


In [10]:
data = data.drop(['check', 'Record_id','y_pred', 'Reason'], axis =1)
data.head()

,y_act,y_Arun,Attribute_name,Total_val,num of dist_val,Unnamed: 9,Num of nans,mean,std_dev,min_val,max_val,sample_1,sample_2,sample_3,sample_4,sample_5
0,Usable directly categorical,Usable directly categorical,Area Code,21477,174,0.810169,0,125.449411,72.866452,1.0,276.0,2,3,4,7,8
1,Context_specific,Usable directly categorical,Item Code,21477,117,0.544769,0,2694.211529,148.969938,2511.0,2961.0,2511,2805,2513,2514,2517
2,Context_specific,Usable directly categorical,Element Code,21477,2,0.009312,0,5211.687154,146.816661,5142.0,5521.0,5142,5521,5142,5142,5521
3,Usable directly categorical,Usable directly categorical,Element,21477,2,0.009312,0,0.000000,0.000000,0.0,0.0,Food,Feed,Food,Food,Food
4,Context_specific,Context_specific,Loan Theme ID,15736,718,4.562786,0,0.000000,0.000000,0.0,0.0,a1050000000slfi,a10500000068jPe,a1050000002X1Uu,a1050000007VvXr,a1050000000weyk


In [11]:
def get_castability(dat):
    sample_values = dat['sample_1':'sample_5']
    #make sure the value you are avaluating is not nan
    castability = 0
    for values in sample_values:
        if pd.isnull(values) == False and values.__class__.__name__ == 'str':
            try:
                castability = str(type(eval(values)))
                castability = 1
                break
            except:
                #print('passing')
                pass
    return castability  

def get_extractability(dat, cast):
    sample_values = dat['sample_1':'sample_5']
    #make sure the value you are avaluating is not nan
    extractability = 0
    for values in sample_values:
        if pd.isnull(values) == False and values.__class__.__name__ == 'str' and cast == 0:
            try:
                #it will faile when you have no numbers or if you have two numbers seperated by space
                float(re.sub('[^0-9\. ]', ' ',values))
                #print('yes')
                extractability = 1
                break
            except:
                pass
    return extractability 

def get_len(dat):
    sample_values = dat['sample_1':'sample_5']
    #make sure the value you are avaluating is not nan
    length = 0
    for values in sample_values:
        if pd.isnull(values) == False:
            length = len(str(values).split()) + length
    return length/5.0 

def bag_of_words_extraction(name_keys):
    vectoriser = DictVectorizer()
    word_vec = attr_bagofwords(name_keys)
    X = vectoriser.fit_transform(Counter(lst) for lst in word_vec)
    b_of_words = X.A
    dict_of_words = vectoriser.vocabulary_
    return b_of_words, dict_of_words
    
    
def attr_bagofwords(name_ke):
    b_words = []
    d = enchant.Dict("en_US")
    for name in tqdm(name_ke):
            name = name.split()
            temp = []
            for i in name:
                if d.check(i.lower()):
                    temp.extend(i.lower())
                else:
                    temp.extend(d.suggest(i.lower()))
            b_words.append(temp)
    return b_words

In [12]:
# get castability, extractability, bag of words, get class type
castability = []
extractability = []
bag_of_words = []
len_val = []
for i in tqdm(range(len(data))):
    castability.append(get_castability(data.loc[i]))
    extractability.append(get_extractability(data.loc[i], castability[i]))
    len_val.append(get_len(data.loc[i]))
data['castability'] = castability
data['extractability'] = extractability
data['len_val'] = len_val  

100%|████████████████████████████████████████████████████████████████████████████| 1080/1080 [00:00<00:00, 1093.79it/s]


In [13]:
data.keys()

Index(['y_act', 'y_Arun', 'Attribute_name', 'Total_val', 'num of dist_val',
       'Unnamed: 9', 'Num of nans', 'mean', 'std_dev', 'min_val', 'max_val',
       'sample_1', 'sample_2', 'sample_3', 'sample_4', 'sample_5',
       'castability', 'extractability', 'len_val'],
      dtype='object')

In [14]:
# replace nan to unk
data['Attribute_name'] = ['unk' if pd.isnull(i) else i for i in data['Attribute_name']]
bag_of_words, dict_of_words = bag_of_words_extraction(data['Attribute_name'])

NameError: name 'enchant' is not defined

In [9]:
newlist = sorted(dict_of_words, key=dict_of_words.get)
temp_df = pd.DataFrame(bag_of_words, columns = newlist)

In [10]:
data = pd.concat([data, temp_df], axis =1)
del temp_df
data.head()

,Attribute_name,Num of nans,Total_val,max_val,mean,min_val,num of dist_val,sample_1,sample_2,sample_3,...,wt e,wt o,x,xv,y,you,yr,z,zip code,zipped
0,Area Code,0,21477,276.0,125.449411,1.0,174,2,3,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Item Code,0,21477,2961.0,2694.211529,2511.0,117,2511,2805,2513,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Element Code,0,21477,5521.0,5211.687154,5142.0,2,5142,5521,5142,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Element,0,21477,0.0,0.000000,0.0,2,Food,Feed,Food,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Loan Theme ID,0,15736,0.0,0.000000,0.0,718,a1050000000slfi,a10500000068jPe,a1050000002X1Uu,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
data.to_csv('data_for_ML.csv', index = None)